In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
from numpy import newaxis
import scipy.sparse as sps
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt

In [ ]:
from pyfem.topo import Interval
from pyfem.poly import gll_points
from pyfem.sem import SEMhat
from pyfem.poly import eval_lagrange_d0 as eval_phi1d

In [ ]:
from poly import eval_P
from utils import minmod

## Build Mesh and Operators

In [ ]:
order = 1
periodic = True
semh = SEMhat(order)

N = 100
n_dofs = (order+1)*N

In [ ]:
L = 2.0*4

vertices  = np.linspace(0, L, N+1)
EtoV      = np.zeros((N, 2), dtype=np.int)
EtoV[:,0] = np.arange(N)
EtoV[:,1] = np.arange(N)+1
if periodic:
    EtoV[-1,-1] = EtoV[0,0]

topo  = Interval()
xq = topo.ref_to_phys(vertices[EtoV], semh.xgll)
jacb_det = topo.calc_jacb(vertices[EtoV])[0]
dx = np.min(xq[0,1:]-xq[0,:-1])

# Restriction operator
# Removes contribution to dirch points from flux operators
R = sps.eye(n_dofs).tocsr()
if not periodic:
    R[0,0] = R[-1,-1] = 0.0

In [ ]:
# Make elem to dof map
EtoD = np.arange(N*(order+1))
EtoD = EtoD.reshape((N, -1))

dof_phys = xq.ravel()

# Averaging operator
rows = EtoD[:,[0,-1]].ravel()
cols = EtoV.ravel()
vals = np.ones_like(cols)

FtoD = sps.coo_matrix((vals, (rows, cols))).tocsr()
AVG = FtoD.dot(FtoD.T)/2.0

# Extract face DOFS
vals = np.ones(len(rows))
FD = sps.coo_matrix((vals, (rows, rows))).tocsr()
# Set face signs
vals[::2] = -1
SD = sps.coo_matrix((vals, (rows, rows))).tocsr()

# Jump operator
JUMP = FtoD.dot(SD.dot(FtoD).T)

In [ ]:
# Build Advection operator
C = sps.kron(sps.eye(N), semh.Ch).tocsr()

# Build full elemental mass matrix
x, w = topo.get_quadrature(order+1)
P = eval_phi1d(semh.xgll, x).T
G = sps.dia_matrix((w, 0), shape=(len(x), len(x)))
Bf = P.T.dot(G.dot(P))*jacb_det
Bfinv = np.linalg.inv(Bf)

# Using trick from book
V    = eval_P(order, semh.xgll).T
Vinv = np.linalg.inv(V)
Minv = V.dot(V.T)/jacb_det
Binv = sps.kron(sps.eye(N), Minv).tocsr()

## Problem Setup

In [ ]:
# # Euler -- Sod's Shock Tube

# use_filter = False

# gamma = 7.0/5.0
# def calc_flux(u):
    
#     f = np.zeros_like(u)
#     p = (gamma-1)*(u[:,2]-.5*u[:,1]**2/u[:,0])
#     f[:,0] = u[:,1]
#     f[:,1] = u[:,1]**2/u[:,0]+p
#     f[:,2] = (u[:,2]+p)*u[:,1]/u[:,0]
    
#     return f

# def calc_p(u):
#     p = (gamma-1)*(u[:,2]-.5*u[:,1]**2/u[:,0])
#     return p

# def calc_eig(u):
#     p = calc_p(u)
#     eig  = np.abs(u[:,1]/u[:,0])
#     eig += np.sqrt(gamma*p/u[:,0])
#     return eig

# # Set up classical Sod's shock tube
# u0 = np.zeros((n_dofs, 3))
# u0[:,0] = 1.0
# u0[:,0][dof_phys>=0.5] = 0.125
# u0[:,1] = 0.0
# u0[:,2] = 1.0
# u0[:,2][dof_phys>=0.5] = 0.1
# u0[:,2] /= (gamma-1)

# np.max(calc_eig(u0))

In [ ]:
# # Isothermal Euler -- Gaussian bump

# a = 1.0
# def calc_flux(u):
    
#     f = np.zeros_like(u)
#     f[:,0] = u[:,1]
#     f[:,1] = u[:,1]**2/u[:,0]+a*a*u[:,0]
#     f[:,2] = 0.0
    
#     return f

# def calc_eig(u):
#     return a+np.abs(u[:,1]/u[:,0])

# u0 = np.zeros((n_dofs, 3))
# aa  = 100.*(dof_phys-L/2.0)**2
# u0[:,0] = 2.0+.2*np.exp(-aa)
# ue = u0.copy()

# np.max(calc_eig(u0))

In [ ]:
# # Shallow water -- Gaussian bump

# a = 1.0
# def calc_flux(u):
    
#     f = np.zeros_like(u)
#     f[:,0] = u[:,1]
#     f[:,1] = u[:,1]**2/u[:,0]+.5*a*u[:,0]**2
#     f[:,2] = 0.0
    
#     return f

# def calc_eig(u):
#     return u[:,1]/u[:,0]+np.sqrt(a*u[:,0])

# u0 = np.zeros((n_dofs, 3))
# aa  = 100.*(dof_phys-L/2.0)**2
# u0[:,0] = 1.0+.5*np.exp(-aa)
# ue = u0.copy()

# np.max(calc_eig(u0))

In [ ]:
# # Burger -- Simple Shock

# def calc_flux(u):
    
#     return u*u/2.0


# def calc_eig(u):

#     return np.abs(u[:,0])

# ul = 2.0
# ur = 1.0
# s  = (calc_flux(ur)-calc_flux(ul))/(ur-ul)

# # u0 = np.ones((n_dofs, 3))*ur
# # u0[dof_phys<=L/2.0,:] = ul

# # ue = np.ones_like(u0)*ur
# # ue[dof_phys<=L/2.0+s*dt*nt,:] = ul

# u0 = np.zeros((n_dofs, 3))
# aa  = 100.*(dof_phys-L/4.0)**2
# u0[:,0] = 1.0+1.0*np.exp(-aa)
# ue = u0.copy()

In [ ]:
# Linear Advection -- Gaussian bump

a = 1.0
def calc_flux(u):
    return a*u

def calc_eig(u):
    return a

u0 = np.zeros((n_dofs, 3))
aa  = 100.*(dof_phys-L/2.0)**2
u0[:,0] = np.exp(-aa)
ue = u0.copy()

np.max(calc_eig(u0))

## Compute solution

In [ ]:
# Slope limiter

# elem midpoints
x0 = (xq[:,0]+xq[:,-1])/2.0
# elem widths (assuming uniform)
h  = (xq[:,-1]-xq[:,0])[0]

# Remove scale factors built into V
# (b.c. of the def used in nodal-dg)
nv  = np.arange(order+1)
gam = 2.0/(2.0*nv+1)
G = sps.dia_matrix((1.0/np.sqrt(gam),0),
                  shape=Vinv.shape)
G = G.dot(Vinv)
    
def slope_limit(U):
    """ MUSCL Limiter
    """
    
    if not periodic:
        u = U
    else:
        u = np.zeros(U.shape[0]+2*(order+1))
        u[(order+1):-(order+1)] = U
        
    us = u.reshape((-1,order+1)).T
    if periodic:
        us[:,0] = us[:,-2]
        us[:,-1] = us[:,1]
        
    avg, slope = G.dot(us)[[0,1]]
    
    # The two comes from the domain of Legendre polys
    slope *= 2.0/h
    u = u.reshape((-1,order+1))
    
    h2 = h/2.0
    m = minmod(slope[1:-1],
               (avg[2:]-avg[1:-1])/h2,
               (avg[1:-1]-avg[:-2])/h2)
    
    # xq has shape (n_elem, n_dof_per_elem)
    # This is why the rest of the arrays need to use newaxis
    if periodic:
        u[1:-1] = avg[1:-1,newaxis]+(xq-x0[:,newaxis])*m[:,newaxis]
    else:
        u[1:-1] = avg[1:-1,newaxis]+(xq-x0[:,newaxis])[1:-1]*m[:,newaxis]
    
    if periodic:
        U[:] = u[1:-1].reshape(U.shape)
        
def apply_limiter(u):
    slope_limit(u[:,0])
    slope_limit(u[:,1])
    slope_limit(u[:,2])

def set_bc(u):
    if not periodic:
        u[[0,-1],:] = u0[[0,-1],:]

In [ ]:
# # Integrate with RK4
# u  = u0.copy()
# f0 = calc_flux(u0[[0,-1],:])*0
# lambda_max = np.max(calc_eig(u0))

# FLUX = AVG+(lambda_max/2.0)*JUMP
# F = Binv.dot(-C+R.dot(SD.dot(FD-FLUX)))
# g = lambda u: F.dot(calc_flux(u))
    
# meig = np.zeros(nt)

# for k in range(nt):
        
#     lambda_max = np.max(calc_eig(u))
#     dt = CFL*dx/lambda_max
    
#     FLUX = AVG+(lambda_max/2.0)*JUMP
#     F = Binv.dot(-C+R.dot(SD.dot(FD-FLUX)))
#     g = lambda u: F.dot(calc_flux(u))
        
#     k1 = g(u)
#     k1[[0,-1],:] = f0
#     apply_limiter(k1)
    
#     k2 = g(u+(dt/2.0)*k1)
#     k2[[0,-1],:] = f0
#     apply_limiter(k2)
    
#     k3 = g(u+(dt/2.0)*k2)
#     k3[[0,-1],:] = f0
#     apply_limiter(k3)
    
#     k4 = g(u+dt*k3)
#     k4[[0,-1],:] = f0
#     apply_limiter(k4)
    
#     u = u+(dt/6.0)*(k1+2*k2+2*k3+k4)
#     u[[0,-1],:] = u0[[0,-1],:]
#     apply_limiter(u)

#     meig[k] = np.max(calc_eig(u))

In [ ]:
nt = 150
CFL = 0.5

In [ ]:
# Integrate with SSP-RK3
u  = u0.copy()
f0 = calc_flux(u0[[0,-1],:])
lambda_max = np.max(calc_eig(u0))

FLUX = AVG+(lambda_max/2.0)*JUMP
F = Binv.dot(-C+R.dot(SD.dot(FD-FLUX)))
g = lambda u: F.dot(calc_flux(u))
    
meig = np.zeros(nt)

for k in range(nt):
        
    lambda_max = np.max(calc_eig(u))
    dt = CFL*dx/lambda_max
    
    FLUX = AVG+(lambda_max/2.0)*JUMP
    F = Binv.dot(-C+R.dot(SD.dot(FD-FLUX)))
    g = lambda u: F.dot(calc_flux(u))
        
    v1 = u+dt*g(u)
    set_bc(v1)
    apply_limiter(v1)
    
    v2 = .25*(3*u+v1+dt*g(v1))
    set_bc(v2)
    apply_limiter(v2)
    
    u = (u+2*v2+2*dt*g(v2))/3.0
    set_bc(u)
    apply_limiter(u)

    meig[k] = np.max(calc_eig(u))

In [ ]:
k = -1 if periodic else n_dofs

plt.figure()
plt.plot(dof_phys[:k], u[:,0][:k])
plt.plot(dof_phys[:k], ue[:,0][:k], 'g--')
plt.ylabel('$\\rho$', size=16)

plt.figure()
plt.plot(dof_phys[:k], u[:,1][:k])
plt.plot(dof_phys[:k], ue[:,1][:k], 'g--')
plt.ylabel('$u$', size=16)

# plt.figure()
# plt.plot(dof_phys[:k], u[:,2][:k])
# plt.plot(dof_phys[:k], ue[:,2][:k], 'g--')
# plt.ylabel('$E$', size=16)

In [ ]:
np.sum(CFL*dx/meig)*np.max(meig)

In [ ]:
plt.plot(CFL*dx/meig)